In [1]:
from keras.models import load_model
import pickle
import cv2
import numpy as np

In [2]:
loaded_model=load_model('FruitClassifier.h5')

In [3]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 48)        3648      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 92, 92, 112)       134512    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 88, 88, 112)       313712    
_________________________________________________________________
flatten (Flatten)            (None, 867328)            0         
_________________________________________________________________
dense (Dense)                (None, 48)                41631792  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 49        
Total params: 42,083,713
Trainable params: 42,083,713
Non-trainable params: 0
____________________________________________

In [8]:
img=cv2.imread('apple_fresh.png')
cv2.imshow('Image', img)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [9]:
def predict_class(img):
    img=cv2.resize(img, (100,100))
    img=img/255
    Img=img.reshape(1,100,100,3)
    for i in loaded_model.predict(Img):
        for j in i:
            if i >= 0.5:
            
                return 'Rotten'
            else:
                return 'Fresh'

In [10]:
predict_class(img)

'Fresh'

In [11]:
if predict_class(img) == 'Rotten':
    cv2.rectangle(img, (10,10), (img.shape[1]-10, img.shape[0]-10), (0,0,255), 2)
    cv2.putText(img,'Rotten', (15,30), cv2.FONT_HERSHEY_COMPLEX_SMALL,1, (0,0,255), 1)
    cv2.imshow('classified', img)
    cv2.waitKey(10000)
    cv2.destroyAllWindows()
elif predict_class(img) == 'Fresh':
    cv2.rectangle(img, (10,10), (img.shape[1]-10, img.shape[0]-10), (0,255,0), 2)
    cv2.putText(img,'Fresh', (15,25), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0,255,0), 1)
    cv2.imshow('classified', img)
    cv2.waitKey(10000)
    cv2.destroyAllWindows()

In [206]:
 cv2.imwrite('rotten_result.png', img)

True